In [1]:
cd '/scratch/nf33/rd7475/ohw24_proj/ohw24_proj_micronekton_img_pipeline_au'

/scratch/nf33/rd7475/ohw24_proj/ohw24_proj_micronekton_img_pipeline_au


### Source info DataFrame

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import re
import time

def extract_data(pattern, file_paths):
    """Extract unique numeric data from file paths based on regex pattern."""
    data = []
    seen = set()
    for path in file_paths:
        path = str(path)
        match = re.search(pattern, path)
        if match:
            num = int(match.group(1))  # Change to float(match.group(1)) if needed
            if num not in seen:
                seen.add(num)
                data.append(num)
    return sorted(data)

####################################################################################

# # Define source path
# source_path = Path('/Volumes/csiro_data/IN2020_V08/PLAOS/Data/')

# # Get relevant file paths
# relevent_paths = list(source_path.rglob('Station_*/**/OBL*.JPG'))

####################################################################################

import pickle
with open('relevent_paths.pkl', 'rb') as file:
    relevent_paths = pickle.load(file)

####################################################################################

relevent_paths_str = []
for p in relevent_paths:
    relevent_paths_str.append(str(p))

# Extract station numbers
station_numbers = extract_data(pattern=r'Station_(\d+)', file_paths=relevent_paths)

# CAP NUMBER OF STATIONS FOR THE MOMENT!
station_numbers = [f'{num:02}' for num in np.arange(1, 3)]

# Prepare lists to store data
img_input_path = []
S = []
I = []

# Process each station
for station in station_numbers:

    station_paths = []
    for path in relevent_paths_str:
        # Format the station number as a two-digit string
        station_str = f'{station:02}'
        # Construct the station path with the formatted station number
        station_path = f'Station_{station_str}'
        # Check if the formatted station path is in the path
        if station_path in path:
            station_paths.append(path)

    # Extract image numbers for the current station
    img_numbers = extract_data(pattern=r'OBL(\d+)', file_paths=station_paths)

    # Process each image number
    for img_number in img_numbers:

        img_number = f'{img_number:05}'

        img_path = [s for s in station_paths if 'OBL'+img_number in s]

        img_input_path.append(str(img_path[0]))
        S.append(station)
        I.append('OBL'+img_number)

M = img_input_path

# Create a DataFrame
df = pd.DataFrame({
    'Station': S,
    'Photo': I,
    'Source': M
})

source_info = df
source_info

,Station,Photo,Source
0,01,OBL00001,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
1,01,OBL00002,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
2,01,OBL00003,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
3,01,OBL00004,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
4,01,OBL00005,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
...,...,...,...
5447,02,OBL09995,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
5448,02,OBL09996,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
5449,02,OBL09997,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
5450,02,OBL09998,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...


### Exif info DataFrame

In [3]:
for station in station_numbers:

    exif_info_station = pd.read_csv('IN2020_V08_PLAOS_station_'+str(int(station))+'_oblique_exif.txt', header=None, names=['Photo', 'Date', 'Time', 'AMPM', 'Photo_number'])
    exif_info_station = exif_info_station.iloc[1:].reset_index(drop=True)
    exif_info_station.insert(0, 'Station', station)

    if station == station_numbers[0]:
        exif_info = exif_info_station

    exif_info = pd.concat([exif_info, exif_info_station], ignore_index=True)

exif_info


,Station,Photo,Date,Time,AMPM,Photo_number
0,01,OBL09618,5/12/2020,06:54:03,AM,1
1,01,OBL09619,5/12/2020,06:54:05,AM,2
2,01,OBL09620,5/12/2020,06:54:07,AM,3
3,01,OBL09621,5/12/2020,06:54:09,AM,4
4,01,OBL09622,5/12/2020,06:54:11,AM,5
...,...,...,...,...,...,...
5818,02,OBL01531,5/12/2020,13:09:10,PM,1917
5819,02,OBL01532,5/12/2020,13:09:12,PM,1918
5820,02,OBL01533,5/12/2020,13:09:14,PM,1919
5821,02,OBL01534,5/12/2020,13:09:16,PM,1920


### Combine relevant info for each image

In [4]:
import_df = pd.DataFrame()

for index, img_info in exif_info.iterrows():
    station = img_info.Station
    photo = img_info.Photo
    
    # Filter the source_info DataFrame to match the current row
    img_source_info = source_info[(source_info.Station == station) & (source_info.Photo == photo)]
    
    if not img_source_info.empty:
        # Convert both rows to dictionaries
        img_info_dict = img_info.to_dict()
        img_source_info_dict = img_source_info.iloc[0].to_dict()
        
        # Combine dictionaries, img_source_info_dict will overwrite img_info_dict if there are conflicts
        combined_dict = {**img_info_dict, **img_source_info_dict}
        
        # Convert the combined dictionary to a DataFrame
        new_row_df = pd.DataFrame([combined_dict])
        
        # Append the new row DataFrame to import_df
        import_df = pd.concat([import_df, new_row_df], ignore_index=True)

import_df.insert(len(import_df.columns) - 1, 'Depth', np.nan)

import_df['Date'] = pd.to_datetime(import_df['Date']).dt.strftime('%m/%d/%Y')

import_df


,Station,Photo,Date,Time,AMPM,Photo_number,Depth,Source
0,01,OBL09618,05/12/2020,06:54:03,AM,1,NaN,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
1,01,OBL09619,05/12/2020,06:54:05,AM,2,NaN,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
2,01,OBL09620,05/12/2020,06:54:07,AM,3,NaN,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
3,01,OBL09621,05/12/2020,06:54:09,AM,4,NaN,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
4,01,OBL09622,05/12/2020,06:54:11,AM,5,NaN,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
...,...,...,...,...,...,...,...,...
5818,02,OBL01531,05/12/2020,13:09:10,PM,1917,NaN,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
5819,02,OBL01532,05/12/2020,13:09:12,PM,1918,NaN,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
5820,02,OBL01533,05/12/2020,13:09:14,PM,1919,NaN,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...
5821,02,OBL01534,05/12/2020,13:09:16,PM,1920,NaN,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...


### Find Depth for each Image

In [5]:
import pandas as pd
from datetime import datetime, timedelta

def round_to_nearest_second(time_str):
    dt = datetime.strptime(time_str, '%H:%M:%S:%f')
    rounded_seconds = round(dt.second + dt.microsecond / 1_000_000)
    new_dt = dt.replace(second=0, microsecond=0) + timedelta(seconds=rounded_seconds)
    return new_dt.strftime('%H:%M:%S')

depth_info = pd.read_csv('IN2020_V08_date_time_depth_20201129-20201209-op-test-03.txt', 
                         header=None, 
                         names=['XXX', 'Date', 'Time', 'Depth']).iloc[:, 1:]

depth_info = depth_info.iloc[1:].reset_index(drop=True)
depth_info['Date'] = pd.to_datetime(depth_info['Date'], format='%Y%m%d').dt.strftime('%d/%m/%Y')


depth_info['Time'] = depth_info['Time'].apply(lambda x: f"{int(x * 100):08d}")
t_str_new = [round_to_nearest_second(f"{t[:2]}:{t[2:4]}:{t[4:6]}:{t[6:]}") for t in depth_info['Time']]
depth_info['Time'] = t_str_new

depth_info


,Date,Time,Depth
0,29/11/2020,23:20:29,4.7
1,29/11/2020,23:20:30,3.6
2,29/11/2020,23:20:31,3.6
3,29/11/2020,23:20:32,3.2
4,29/11/2020,23:20:33,3.2
...,...,...,...
61634,09/12/2020,10:37:43,4.3
61635,09/12/2020,10:37:44,3.2
61636,09/12/2020,10:37:44,3.2
61637,09/12/2020,10:37:46,1.4


In [6]:
import pandas as pd
from datetime import datetime
import numpy as np
from scipy.spatial import cKDTree

depth_info['Datetime'] = depth_info.apply(lambda row: datetime.strptime(f"{row['Date']} {row['Time']}", "%d/%m/%Y %H:%M:%S"), axis=1)
depth_info['Timestamp'] = depth_info['Datetime'].astype(np.int64) // 10**9

tree = cKDTree(depth_info[['Timestamp']].values)

import_df['Datetime'] = import_df.apply(lambda row: datetime.strptime(f"{row['Date']} {row['Time']}", "%d/%m/%Y %H:%M:%S"), axis=1)
import_df['Timestamp'] = import_df['Datetime'].astype(np.int64) // 10**9

distances, indices = tree.query(import_df[['Timestamp']].values, k=1)

import_df['Depth'] = depth_info.iloc[indices.flatten()]['Depth'].values


### Finalise DataFrame

In [8]:
final_import_df = import_df.drop(columns=['Date', 'Time', 'Timestamp', 'Photo_number', 'AMPM'])
cols = final_import_df.columns.tolist()
last_col = cols.pop()
cols.insert(2, last_col)
final_import_df = final_import_df[cols]

final_import_df['OutSource'] = final_import_df.astype(str).apply('/'.join, axis=1) \
                              .str.replace('//', '/') \
                              .str.replace(' ', 'T') \
                              .apply(lambda x: f'/IN2020_V09/{x}')

final_import_df

,Station,Photo,Datetime,Depth,Source,OutSource
0,01,OBL09618,2020-12-05 06:54:03,6.9,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...,/IN2020_V09/01/OBL09618/2020-12-05T06:54:03/6....
1,01,OBL09619,2020-12-05 06:54:05,6.2,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...,/IN2020_V09/01/OBL09619/2020-12-05T06:54:05/6....
2,01,OBL09620,2020-12-05 06:54:07,6.5,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...,/IN2020_V09/01/OBL09620/2020-12-05T06:54:07/6....
3,01,OBL09621,2020-12-05 06:54:09,5.4,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...,/IN2020_V09/01/OBL09621/2020-12-05T06:54:09/5....
4,01,OBL09622,2020-12-05 06:54:11,5.8,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...,/IN2020_V09/01/OBL09622/2020-12-05T06:54:11/5....
...,...,...,...,...,...,...
5818,02,OBL01531,2020-12-05 13:09:10,5.8,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...,/IN2020_V09/02/OBL01531/2020-12-05T13:09:10/5....
5819,02,OBL01532,2020-12-05 13:09:12,6.2,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...,/IN2020_V09/02/OBL01532/2020-12-05T13:09:12/6....
5820,02,OBL01533,2020-12-05 13:09:14,5.4,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...,/IN2020_V09/02/OBL01533/2020-12-05T13:09:14/5....
5821,02,OBL01534,2020-12-05 13:09:16,6.9,/Volumes/csiro_data/IN2020_V08/PLAOS/Data/Stat...,/IN2020_V09/02/OBL01534/2020-12-05T13:09:16/6....


## Function

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import re
import time
from datetime import datetime, timedelta
from scipy.spatial import cKDTree

####################################################################################

# # Define source path
# source_path = Path('/Volumes/csiro_data/IN2020_V08/PLAOS/Data/')

# # Get relevant file paths
# relevent_paths = list(source_path.rglob('Station_*/**/OBL*.JPG'))

####################################################################################

import pickle
with open('relevent_paths.pkl', 'rb') as file:
    relevent_paths = pickle.load(file)

####################################################################################


def pipeline(relevent_paths):
    
    def extract_data(pattern, file_paths):
        """Extract unique numeric data from file paths based on regex pattern."""
        data = []
        seen = set()
        for path in file_paths:
            path = str(path)
            match = re.search(pattern, path)
            if match:
                num = int(match.group(1))  
                if num not in seen:
                    seen.add(num)
                    data.append(num)
        return sorted(data)
    

    relevent_paths_str = []
    for p in relevent_paths:
        relevent_paths_str.append(str(p))
    
    # Extract station numbers
    station_numbers = extract_data(pattern=r'Station_(\d+)', file_paths=relevent_paths)
    
    # CAP NUMBER OF STATIONS FOR THE MOMENT!
    station_numbers = [f'{num:02}' for num in np.arange(1, 3)]
    
    # Prepare lists to store data
    img_input_path = []
    S = []
    I = []
    
    # Process each station
    for station in station_numbers:
    
        station_paths = []
        for path in relevent_paths_str:
            # Format the station number as a two-digit string
            station_str = f'{station:02}'
            # Construct the station path with the formatted station number
            station_path = f'Station_{station_str}'
            # Check if the formatted station path is in the path
            if station_path in path:
                station_paths.append(path)
    
        # Extract image numbers for the current station
        img_numbers = extract_data(pattern=r'OBL(\d+)', file_paths=station_paths)
    
        # Process each image number
        for img_number in img_numbers:
    
            img_number = f'{img_number:05}'
    
            img_path = [s for s in station_paths if 'OBL'+img_number in s]
    
            img_input_path.append(str(img_path[0]))
            S.append(station)
            I.append('OBL'+img_number)
    
    # Create a DataFrame
    source_info = pd.DataFrame({
        'Station': S,
        'Photo': I,
        'Source': img_input_path
    })

    ################
    
    for station in station_numbers:
    
        exif_info_station = pd.read_csv('IN2020_V08_PLAOS_station_'+str(int(station))+'_oblique_exif.txt', header=None, names=['Photo', 'Date', 'Time', 'AMPM', 'Photo_number'])
        exif_info_station = exif_info_station.iloc[1:].reset_index(drop=True)
        exif_info_station.insert(0, 'Station', station)
    
        if station == station_numbers[0]:
            exif_info = exif_info_station
    
        exif_info = pd.concat([exif_info, exif_info_station], ignore_index=True)
    
    ###############
    
    import_df = pd.DataFrame()
    
    for index, img_info in exif_info.iterrows():
        station = img_info.Station
        photo = img_info.Photo
        
        # Filter the source_info DataFrame to match the current row
        img_source_info = source_info[(source_info.Station == station) & (source_info.Photo == photo)]
        
        if not img_source_info.empty:
            # Convert both rows to dictionaries
            img_info_dict = img_info.to_dict()
            img_source_info_dict = img_source_info.iloc[0].to_dict()
            
            # Combine dictionaries, img_source_info_dict will overwrite img_info_dict if there are conflicts
            combined_dict = {**img_info_dict, **img_source_info_dict}
            
            # Convert the combined dictionary to a DataFrame
            new_row_df = pd.DataFrame([combined_dict])
            
            # Append the new row DataFrame to import_df
            import_df = pd.concat([import_df, new_row_df], ignore_index=True)
    
    import_df.insert(len(import_df.columns) - 1, 'Depth', np.nan)
    
    import_df['Date'] = pd.to_datetime(import_df['Date']).dt.strftime('%m/%d/%Y')
    
    
    ###############
    
    
    def round_to_nearest_second(time_str):
        dt = datetime.strptime(time_str, '%H:%M:%S:%f')
        rounded_seconds = round(dt.second + dt.microsecond / 1_000_000)
        new_dt = dt.replace(second=0, microsecond=0) + timedelta(seconds=rounded_seconds)
        return new_dt.strftime('%H:%M:%S')
    
    depth_info = pd.read_csv('IN2020_V08_date_time_depth_20201129-20201209-op-test-03.txt', 
                             header=None, 
                             names=['XXX', 'Date', 'Time', 'Depth']).iloc[:, 1:]
    
    depth_info = depth_info.iloc[1:].reset_index(drop=True)
    depth_info['Date'] = pd.to_datetime(depth_info['Date'], format='%Y%m%d').dt.strftime('%d/%m/%Y')
    
    
    depth_info['Time'] = depth_info['Time'].apply(lambda x: f"{int(x * 100):08d}")
    t_str_new = [round_to_nearest_second(f"{t[:2]}:{t[2:4]}:{t[4:6]}:{t[6:]}") for t in depth_info['Time']]
    depth_info['Time'] = t_str_new
    
    ######################
    
    depth_info['Datetime'] = depth_info.apply(lambda row: datetime.strptime(f"{row['Date']} {row['Time']}", "%d/%m/%Y %H:%M:%S"), axis=1)
    depth_info['Timestamp'] = depth_info['Datetime'].astype(np.int64) // 10**9
    
    tree = cKDTree(depth_info[['Timestamp']].values)
    
    import_df['Datetime'] = import_df.apply(lambda row: datetime.strptime(f"{row['Date']} {row['Time']}", "%d/%m/%Y %H:%M:%S"), axis=1)
    import_df['Timestamp'] = import_df['Datetime'].astype(np.int64) // 10**9
    
    distances, indices = tree.query(import_df[['Timestamp']].values, k=1)
    
    import_df['Depth'] = depth_info.iloc[indices.flatten()]['Depth'].values
    
    
    ######################
    
    final_import_df = import_df.drop(columns=['Date', 'Time', 'Timestamp', 'Photo_number'])
    cols = final_import_df.columns.tolist()
    last_col = cols.pop()
    cols.insert(2, last_col)
    final_import_df = final_import_df[cols]
